In [22]:
import torch

def predict_word_likelihood(sentence, word, model, tokenizer):
    tokenized_text = tokenizer.tokenize(sentence)

    # Mask the word we want to predict
    masked_index = tokenized_text.index(word)
    tokenized_text[masked_index] = '[MASK]'

    # Convert tokenized text to BERT input format
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predict the likelihood of the word using BERT
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]

    # Get the likelihood of the word at the masked position
    likelihood = predictions[0, masked_index, :]

    # Convert likelihood to a probability distribution
    likelihood = torch.softmax(likelihood, dim=0)
    print(likelihood)

      # Get the probability of the word
    word_index = tokenizer.convert_tokens_to_ids([word])[0]
    word_prob = likelihood[word_index].item()
    
    return word_prob

In [18]:
    # Load the BERT model
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("ltgoslo/norbert")

model = AutoModelForMaskedLM.from_pretrained("ltgoslo/norbert")

Some weights of the model checkpoint at ltgoslo/norbert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
print(predict_word_likelihood('Jeg skal en rask tur på byen', 'byen', model, tokenizer))

tensor([1.5384e-08, 2.9364e-08, 9.6249e-09,  ..., 1.1255e-08, 3.0849e-08,
        3.6763e-07])
0.07478123903274536
